In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Section One: Define the Parameters of the Search
To create a search we need to "build" a URL that takes us to a valid results query, this requires taking our base endpoint and attaching on different parameters to help narrow down our search. I'll do my best to explain how each of these parameters works, but unfortunately, there is no formal documentation on this.

Endpoint The endpoint for our EDGAR query is https://www.sec.gov/cgi-bin/browse-edgar if you go to this link without any additional parameters it will be an invalid request.

--------------------------------------------------------------------
### Parameters:

- **action:** (required) By default should be set to getcompany.

- **CIK**: (required) Is the CIK number of the company you are searching.

- **type**: (optional) Allows filtering the type of form. For example, if set to 10-k only the 10-K filings are returned.

- **dateb**: (optional) Will only return the filings before a given date. The format is as follows YYYYMMDD

- **owner:** (required) Is set to exclude by default and specifies ownership. You may also set it to include and only.

- **start:** (optional) Is the starting index of the results. For example, if I have 100 results but want to start at 45 of 100, I would pass 45.

- **state:** (optional) The company's state.

- **filenum:** (optional) The filing number.

- **sic:** (optional) The company's SIC (Standard Industry Classification) identifier
- **output:** (optional) Defines returned data structure as either xml (atom) or normal html.

- **count:** (optional) The number of results you want to see with your request, the max is 100 and if not set it will default to 40.

------------------------------------------------------------------------------
Now that we understand all the parameters let's make a request by defining our endpoint, and then a dictionary of our parameters. Where the key of the dictionary is the parameter name, and the value is the value we want to set for that parameter. Once we've defined these two components we can make our request and parse the response using BeautifulSoup.

In [17]:
# base URL for the SEC EDGAR browser
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

# define our parameters dictionary
param_dict = {'action':'getcompany',
              'CIK':'0000789019',
              'type':'10-Q',
              'dateb':'',
              'owner':'exclude',
              'start':'',
              'output':'',
              'count':'100'}

# request the url, and then parse the response.
response = requests.get(url = endpoint, params = param_dict)
soup = BeautifulSoup(response.content, 'lxml')

# print status code
print(response.status_code)
print(response.url)

200
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000789019&type=10-Q&dateb=&owner=exclude&start=&output=&count=100


In [18]:
# this list will save all interative data links
I_D_links = []

# get all interactive data financial statements
table = soup.find(class_="tableFile2")

rows = table.find_all("tr")[1:]
for row in rows:
    l = []
    s = "interactiveDataBtn"
    if s in row.decode():
        date = row.find_all("td")[3].string
        url = row.find(id=s).get('href')
        l.append("https://www.sec.gov" + url)
        l.append(date)
        I_D_links.append(l)
print(I_D_links)

[['https://www.sec.gov/cgi-bin/viewer?action=view&cik=789019&accession_number=0001564590-20-019706&xbrl_type=v', '2020-04-29'], ['https://www.sec.gov/cgi-bin/viewer?action=view&cik=789019&accession_number=0001564590-20-002450&xbrl_type=v', '2020-01-29'], ['https://www.sec.gov/cgi-bin/viewer?action=view&cik=789019&accession_number=0001564590-19-037549&xbrl_type=v', '2019-10-23'], ['https://www.sec.gov/cgi-bin/viewer?action=view&cik=789019&accession_number=0001564590-19-012709&xbrl_type=v', '2019-04-24'], ['https://www.sec.gov/cgi-bin/viewer?action=view&cik=789019&accession_number=0001564590-19-001392&xbrl_type=v', '2019-01-30'], ['https://www.sec.gov/cgi-bin/viewer?action=view&cik=789019&accession_number=0001564590-18-024893&xbrl_type=v', '2018-10-24'], ['https://www.sec.gov/cgi-bin/viewer?action=view&cik=789019&accession_number=0001564590-18-009307&xbrl_type=v', '2018-04-26'], ['https://www.sec.gov/cgi-bin/viewer?action=view&cik=789019&accession_number=0001564590-18-001129&xbrl_type=v'

In [24]:
F_S_links = []
for l in I_D_links:
    endpoint = l[0]
    l2 = []
    response = requests.get(url = endpoint)
    soup = BeautifulSoup(response.content, 'lxml')
    entries = soup.find("a", string="View Excel Document")
    url = entries.get('href')
    l2.append("https://www.sec.gov" + url)
    l2.append(l[1])
    F_S_links.append(l2)
    print(l2)
    

['https://www.sec.gov/Archives/edgar/data/789019/000156459020019706/Financial_Report.xlsx', '2020-04-29']
['https://www.sec.gov/Archives/edgar/data/789019/000156459020002450/Financial_Report.xlsx', '2020-01-29']
['https://www.sec.gov/Archives/edgar/data/789019/000156459019037549/Financial_Report.xlsx', '2019-10-23']
['https://www.sec.gov/Archives/edgar/data/789019/000156459019012709/Financial_Report.xlsx', '2019-04-24']
['https://www.sec.gov/Archives/edgar/data/789019/000156459019001392/Financial_Report.xlsx', '2019-01-30']
['https://www.sec.gov/Archives/edgar/data/789019/000156459018024893/Financial_Report.xlsx', '2018-10-24']
['https://www.sec.gov/Archives/edgar/data/789019/000156459018009307/Financial_Report.xlsx', '2018-04-26']
['https://www.sec.gov/Archives/edgar/data/789019/000156459018001129/Financial_Report.xlsx', '2018-01-31']
['https://www.sec.gov/Archives/edgar/data/789019/000156459017020171/Financial_Report.xlsx', '2017-10-26']
['https://www.sec.gov/Archives/edgar/data/7890

In [25]:
F_S_links

[['https://www.sec.gov/Archives/edgar/data/789019/000156459020019706/Financial_Report.xlsx',
  '2020-04-29'],
 ['https://www.sec.gov/Archives/edgar/data/789019/000156459020002450/Financial_Report.xlsx',
  '2020-01-29'],
 ['https://www.sec.gov/Archives/edgar/data/789019/000156459019037549/Financial_Report.xlsx',
  '2019-10-23'],
 ['https://www.sec.gov/Archives/edgar/data/789019/000156459019012709/Financial_Report.xlsx',
  '2019-04-24'],
 ['https://www.sec.gov/Archives/edgar/data/789019/000156459019001392/Financial_Report.xlsx',
  '2019-01-30'],
 ['https://www.sec.gov/Archives/edgar/data/789019/000156459018024893/Financial_Report.xlsx',
  '2018-10-24'],
 ['https://www.sec.gov/Archives/edgar/data/789019/000156459018009307/Financial_Report.xlsx',
  '2018-04-26'],
 ['https://www.sec.gov/Archives/edgar/data/789019/000156459018001129/Financial_Report.xlsx',
  '2018-01-31'],
 ['https://www.sec.gov/Archives/edgar/data/789019/000156459017020171/Financial_Report.xlsx',
  '2017-10-26'],
 ['https:/

In [12]:
# the most recent link to the xls doc
F_S_links[0]

'https://www.sec.gov/Archives/edgar/data/1018724/000101872420000021/Financial_Report.xlsx'

In [28]:
# lest download the document using pandas
doc = pd.ExcelFile(F_S_links[0][0])

In [30]:
# the xls have a lot of sheets, lets see one
df1 = pd.read_excel(doc, 'INCOME STATEMENTS')
df1

,"INCOME STATEMENTS - USD ($) shares in Millions, $ in Millions",3 Months Ended,Unnamed: 2,9 Months Ended,Unnamed: 4
0,NaN,"Mar. 31, 2020","Mar. 31, 2019","Mar. 31, 2020","Mar. 31, 2019"
1,Revenue,35021,30571,104982,92126
2,Cost of revenue,10975,10170,33739,32498
3,Gross margin,24046,20401,71243,59628
4,Research and development,4887,4316,14055,12363
5,Sales and marketing,4911,4565,14181,13251
6,General and administrative,1273,1179,3455,3460
7,Operating income,12975,10341,39552,30554
8,"Other income (expense), net",-132,145,62,538
9,Income before income taxes,12843,10486,39614,31092


In [31]:
# to see all sheet names of the doc, we will use the sheet_names atribute
# it's a list wit all sheet names
doc.sheet_names

['Document and Entity Information',
 'INCOME STATEMENTS',
 'COMPREHENSIVE INCOME STATEMENTS',
 'BALANCE SHEETS',
 'BALANCE SHEETS (Parenthetical)',
 'CASH FLOWS STATEMENTS',
 "STOCKHOLDERS' EQUITY STATEMENTS",
 'ACCOUNTING POLICIES',
 'EARNINGS PER SHARE',
 'OTHER INCOME (EXPENSE), NET',
 'INVESTMENTS',
 'DERIVATIVES',
 'INVENTORIES',
 'GOODWILL',
 'INTANGIBLE ASSETS',
 'DEBT',
 'INCOME TAXES',
 'UNEARNED REVENUE',
 'LEASES',
 'CONTINGENCIES',
 "STOCKHOLDERS' EQUITY",
 'ACCUMULATED OTHER COMPREHENSIVE',
 'SEGMENT INFORMATION AND GEOGRAP',
 'ACCOUNTING POLICIES (Policies)',
 'EARNINGS PER SHARE (Tables)',
 'OTHER INCOME (EXPENSE), NET (Ta',
 'INVESTMENTS (Tables)',
 'DERIVATIVES (Tables)',
 'INVENTORIES (Tables)',
 'GOODWILL (Tables)',
 'INTANGIBLE ASSETS (Tables)',
 'DEBT (Tables)',
 'UNEARNED REVENUE (Tables)',
 'LEASES (Tables)',
 "STOCKHOLDERS' EQUITY (Tables)",
 'ACCUMULATED OTHER COMPREHENSI_2',
 'SEGMENT INFORMATION AND GEOGR_2',
 'Accounting Policies - Additiona',
 'Basic and Di

In [ ]:
# it's a list wit all atributes
doc.sheet_names

In [32]:
# lest see one sheet using
pd.read_excel(doc, doc.sheet_names[1])

,"INCOME STATEMENTS - USD ($) shares in Millions, $ in Millions",3 Months Ended,Unnamed: 2,9 Months Ended,Unnamed: 4
0,NaN,"Mar. 31, 2020","Mar. 31, 2019","Mar. 31, 2020","Mar. 31, 2019"
1,Revenue,35021,30571,104982,92126
2,Cost of revenue,10975,10170,33739,32498
3,Gross margin,24046,20401,71243,59628
4,Research and development,4887,4316,14055,12363
5,Sales and marketing,4911,4565,14181,13251
6,General and administrative,1273,1179,3455,3460
7,Operating income,12975,10341,39552,30554
8,"Other income (expense), net",-132,145,62,538
9,Income before income taxes,12843,10486,39614,31092
